In [1]:
import cv2
import os
import re

In [2]:
tmpDir='/tmp'
tmpGeneratedFiles = []

In [3]:
offsetList = ["offset2.cr2", "offset2.cr2", "offset3.cr2"]
darkList = []
flatList = []
imageList = []

In [4]:
def getFileName(fileName, removeExtention=False, removePath=True):
    '''
    get the name of a file from a string (can remove the extention or the complete path)
    
    parameters:
    fileName as a string. The name can include the path
    removeExtention as boolean. Default is False. This parameter remove the extention of the filename. (e.g: 'test.jpg'
    will become 'test')
    removePath as boolean. Default is True. If the complete path is given in the filename is it possible to remove it or not
    
    return value:
    the filename
    
    '''
    if removePath:
        fileName = os.path.basename(fileName)
    if removeExtention:
        re_catchFileName = re.compile('(.*)\..*$')
        catchFileNameResult = re_catchFileName.match(fileName)
        if catchFileNameResult:
            fileName = catchFileNameResult.group(1)

    return fileName

In [5]:
getFileName('/media/jean-phi/EOS_DIGITAL/DCIM/100CANON/IMG_0679.CR2', removeExtention=True, removePath=True)

'IMG_0679'

In [6]:
def uncompressRawFiles(rawFileList):
    '''
    Generate a pgm file from a raw file.
    
    Parameters:
    rawFileList as a list of strings. this parameter is the list of some filenames of camera RAW files.
    
    return value:
    a list of the generated files (with their path)
    '''
    generatedFiles = []
    for file in rawFileList:
        # Extract Raw data in a PGM file with dcraw
        os.system('dcraw -d -4 ' + file)
        generatedFiles.append(getFileName(file, getFileName(file, removeExtention=True, removePath=False) + '.pgm'))
    
    return generatedFiles     

In [10]:
def getRAWdata(rawFileList):
    '''
    Get the image raw data from a list of pgm files
    
    Parameters:
    rawFileList as a list of string. each element of the list is a string of the filname of a pgm file containing the raw data.
    
    Return Value:
    an array of array containing all images RAW data. Each element is an image.
    '''
    offsetStackList = cv2.imread(rawFileList.pop(0), cv2.IMREAD_UNCHANGED)

    for file in offsetList:
        offsetStackList.append(offsetStackList, [cv2.imread(file, cv2.IMREAD_UNCHANGED)], axis=0)
    
    return offsetStackList

In [11]:
def calculateMedian(RAWdataList):
    '''
    Calculate the median of a stack of RAW images
    
    Parameters:
    RAWdataList as an array of array of all images.
    
    Return value:
    The median of each pixel of the image stack
    '''
        
    #offsetStack = np.stack(offsetStackList)
    masterOffset = np.median(offsetStack, axis=0)
    
    return masterOffset

# Generating master offset

The generation of the master offset is just a median of all offset image

In [12]:
RAWoffsetFiles = uncompressRawFiles(offsetList)
RAWoffsetData = getRAWdata(RAWoffsetFiles)
masterOffet = calculateMedian(RAWoffsetData)

AttributeError: 'NoneType' object has no attribute 'append'

# Generating master Dark

In [ ]:
RAWdarkFiles = uncompressRawFiles(darkList)
RAWdarkData = getRAWdata(RAWdarkFiles)
masterDark = calculateMedian(RAWdarkData)

# Generating master Flat

In [ ]:
RAWflatFiles = extractRawData(FlatList)
RAWflatData = getRAWdata(RAWflatFiles)
RAWflatData -= masterOffset
masterFlat = calculateMedian(RAWflatData)
meanFlat = np.mean(masterFlat)

# Calculate images

In [ ]:
# image= (Brute - black)/(PLU - offset)*meanFlat
RAWimageFiles = extractRawData(imageList)
RAWimageData = getRAWdata(RAWimageFiles)
images = ((RAWimageData - masterDark)/(masterFlat - masterOffser))*meanFlat

for i in imageList:
    index = imageList.index(i)
    # Debayreizing the RAW image
    RGB_image = cv2.cvtColor(RAWimageData[index], cv2.COLOR_BayerBG2RGB)
    # Gamma correction
    bitDepth = 2**16
    RGB_image = np.power(RGB_image/bitDepth, 1/2.1)*bitDepth
    tgtFileName = getFileName(i, removeExtention=True, removePath=False) + 'png'
    cv2.imwrite("tgtFileName, RGB_image)

In [ ]:
# remove files: RAWoffsetFiles, RAWdarkFiles, RAWflatFiles and RAWimageFiles